In [2]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "3_clean")

# Decision Trees and Random Forest Models

In [1]:
df= pd.read_csv(os.join(inp, "analysis_tract.csv")).drop("Unnamed: 0", axis = 1)
df.head()

NameError: name 'pd' is not defined

In [17]:
print(df.columns)
df.describe()

Index(['borough', 'cd', 'ct2010', 'zipcode', 'address', 'bldgclass', 'landuse',
       'ownertype', 'ownername', 'lotarea', 'bldgarea', 'numbldgs',
       'numfloors', 'unitsres', 'unitstotal', 'assessland', 'assesstot',
       'exempttot', 'yearbuilt', 'yearalter1', 'yearalter2', 'bbl', 'sanborn',
       'holc_AB', 'holc_CD', 'holc_D', 'num_vac_orders', 'vacate_ind',
       'num_vacate_orders', 'viol_count', 'AL', 'NB', 'average_energy_usage',
       'average_energy_usage_mi'],
      dtype='object')


,cd,ct2010,zipcode,landuse,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,...,holc_CD,holc_D,num_vac_orders,vacate_ind,num_vacate_orders,viol_count,AL,NB,average_energy_usage,average_energy_usage_mi
count,765323.000000,765264.000000,765316.000000,765714.000000,7.654280e+05,7.657140e+05,765428.000000,765428.000000,765708.000000,765712.000000,...,765117.000000,765117.00000,2603.000000,765714.000000,763111.0,765714.000000,765714.000000,765714.000000,765389.000000,765714.000000
mean,357.693954,470.832974,11026.161084,1.423121,3.815063e+03,5.078128e+03,1.324699,2.447578,4.717486,4.981860,...,0.704729,0.22371,1.094891,0.003399,0.0,7.081562,0.002844,0.000876,4.421240,0.976662
std,98.110120,378.922236,463.572291,0.844367,2.317468e+04,3.521141e+04,2.768148,1.693242,42.120376,63.182793,...,0.456165,0.41673,0.438168,0.058206,0.0,50.126042,0.026303,0.010436,850.251643,0.150974
min,101.000000,1.000000,10001.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,307.000000,170.090000,10467.000000,1.000000,2.000000e+03,1.442000e+03,1.000000,2.000000,1.000000,1.000000,...,0.000000,0.00000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
50%,403.000000,363.000000,11223.000000,1.000000,2.500000e+03,2.023000e+03,1.000000,2.000000,2.000000,2.000000,...,1.000000,0.00000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
75%,412.000000,671.000000,11372.000000,2.000000,4.000000e+03,2.928000e+03,2.000000,2.750000,2.000000,2.000000,...,1.000000,0.00000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
max,503.000000,1621.000000,12345.000000,4.000000,1.225343e+07,1.354011e+07,1854.000000,98.000000,19201.000000,36481.000000,...,1.000000,1.00000,15.000000,1.000000,0.0,4825.000000,13.444444,1.333333,400064.000000,1.000000


In [18]:
df= pd.get_dummies(df, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X = df.loc[:, ["landuse", "lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
              "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "viol_count", "AL", "NB",
              "average_energy_usage", "average_energy_usage_mi"]+borocols].fillna(0)
y = df.loc[:, "vacate_ind"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

dt = DecisionTreeClassifier(max_depth=10)
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
accuracy= (y_predict == y_test).value_counts(normalize=True)[True]
accuracy

0.9961038678362318

__Area under the curve__

In [23]:
from sklearn.metrics import roc_auc_score
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
# predict_proba predicts the probability of each class rather than just the most likely class
pred=dt.predict_proba(X_test)[:,1] # predicted probability of y = 1
roc_auc_score(np.array(y_test),pred)

0.5568472977716408

In [26]:
# vs. random chance (only slightly better)
pred=np.random.random(len(X_test))
roc_auc_score(np.array(y_test.apply(int)),pred)

0.4792570536905316

## Tuning

In [27]:
from sklearn.model_selection import GridSearchCV
# This time we'll use max_depth to control the complexity of the tree, still using the same train/test split as above,
# and optimize the parameter value using GridSearchCV.
param_grid = {'max_depth':range(1,20)}
dt=DecisionTreeClassifier()
gr=GridSearchCV(dt,param_grid=param_grid,scoring='roc_auc')
rs=gr.fit(X_train,y_train)
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),rs.predict_proba(X_test)[:,1]))

{'max_depth': 5}
0.9734948545025446


## Feature Importance

In [33]:
dt = DecisionTreeClassifier(max_depth=rs.best_params_['max_depth'])
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 5 most important features in order
Feature_importance.sort_values(by="importance",ascending=False).iloc[:5,:]

,variables,importance
13,viol_count,0.704478
2,bldgarea,0.207128
5,unitsres,0.046483
10,yearbuilt,0.013953
7,assessland,0.012477


## Random Forests

In [34]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_leaf_nodes=10)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test,pred))

0.9681534147606388


In [35]:
param_grid = {'n_estimators':range(1,51,5)}
rf = RandomForestClassifier(n_jobs=-1,max_leaf_nodes=10)
gs = GridSearchCV(rf,param_grid=param_grid,scoring='roc_auc')
rs = gs.fit(X_train,y_train)
pred=rs.predict_proba(X_test)[:,1]
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),pred))

{'n_estimators': 46}
0.9681038129153868


# Naive Bayes?